# Model Experimentation: Hyperparameter Optimization
This notebook conducts experiments on hyperparameter optimization.

## Imports and Setup

In [20]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import sys
sys.path.insert(0, '../utils')
import model_utils
import geoutils

import logging
import warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

SEED = 42

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## File Location

In [21]:
data_dir = "../data/"
input_file = data_dir + '20200415_dataset.csv'

pos_mask_dir = data_dir + 'pos_masks/'
neg_mask_dir = data_dir + 'neg_masks/'
sentinel_dir = data_dir + 'sentinel2/'
false_pos_dir = data_dir + 'false_pos/'

# areas = ['maicao', 'riohacha', 'uribia', 'arauca1', 'cucuta', 'arauquita', 'tibu1']
areas = ['maicao', 'riohacha', 'uribia', 'arauca1', 'cucuta', 'tibu1']

## Load Dataset

In [22]:
data = (pd.read_csv(input_file).reset_index(drop=True)
       .query("area != 5")) # exclude arauquita
print('Data dimensions: {}'.format(data.shape))
data.head()

Data dimensions: (830930, 113)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.15970,0.13735,0.1531,0.18700,0.2090,0.26320,0.30515,0.26965,0.33270,0.0411,...,-0.509745,0.105128,0.239614,0.449106,-0.718433,-0.042537,0.901237,1,3,0
1,0.15970,0.13905,0.1454,0.17845,0.2090,0.26320,0.30515,0.26395,0.33270,0.0411,...,-0.507485,0.105128,0.247826,0.464498,-0.716955,-0.039760,0.911490,1,3,0
2,0.16675,0.14875,0.1589,0.18605,0.2258,0.27945,0.32070,0.28085,0.34520,0.0416,...,-0.524371,0.073259,0.262348,0.446475,-0.722188,-0.033995,0.875915,1,3,0
3,0.16675,0.15240,0.1771,0.23520,0.2258,0.27945,0.32070,0.30740,0.34520,0.0416,...,-0.475631,0.073259,0.308045,0.524245,-0.689591,-0.019520,0.905289,1,3,0
4,0.16095,0.15650,0.1920,0.25320,0.2500,0.30200,0.34190,0.32755,0.36515,0.0421,...,-0.464686,0.059161,0.352879,0.548867,-0.673300,-0.009437,0.875968,1,3,0


## Resample Dataset
Resamples 30,000 negative examples per area.

In [23]:
neg_dist = {'Formal settlement': 0.4, 'Unoccupied land': 0.6}
data = model_utils.resample(data, num_neg_samples=30000, neg_dist=neg_dist, random_state=SEED)
print('Data dimensions: {}'.format(data.shape))
data.head(3)

Data dimensions: (199460, 113)


,B1_2016,B2_2016,B3_2016,B4_2016,B5_2016,B6_2016,B7_2016,B8_2016,B9_2016,B10_2016,...,mndwi_2020,ui_2020,nbi_2020,brba_2020,nbai_2020,mbi_2020,baei_2020,la_guajira,target,area
0,0.16795,0.1330,0.1379,0.1355,0.1571,0.2686,0.3198,0.2773,0.3579,0.0402,...,-0.495702,0.213802,0.122766,0.386654,-0.762164,-0.097429,1.005492,1,2,0
1,0.19330,0.2011,0.2039,0.2299,0.2270,0.2680,0.3094,0.2723,0.3335,0.0401,...,-0.417789,0.102637,0.211351,0.520237,-0.748431,-0.021367,1.030898,1,2,0
2,0.19290,0.1867,0.1810,0.1958,0.1889,0.2415,0.2695,0.2430,0.2892,0.0272,...,-0.188068,0.079984,0.222447,0.746002,-0.641025,-0.030201,1.056083,1,2,0


## Machine Learning Pipeline

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
# from lightgbm import LGBMClassifier
SEED = 42

## Define Features and Target Label

In [25]:
# Get target and feature columns
label = 'target'
features = [column for column in data.columns[:-2]]

# Convert target labels to binary
data[label] = data[label].replace({2:0, 3:0, 4:0})

print('Data dimensions: {}'.format(data.shape))
print('Class distribution:\n{}'.format(data['target'].value_counts()))

Data dimensions: (199460, 113)
Class distribution:
0    178200
1     21260
Name: target, dtype: int64


## Logistic Regression

### Feature Selection: Recursive Feature Eleimination CV

In [26]:
lr = LogisticRegression()
lr_best_features = model_utils.rfecv_feature_selection(
    lr, data, features, label, scoring='f1', step=5, verbose=0
)
print(lr_best_features)

['B10_2016', 'B12_2016', 'nbi_2016', 'B1_2017', 'B2_2017', 'mndwi_2017', 'ui_2017', 'B1_2019', 'B2_2019', 'B3_2019', 'B7_2019', 'B9_2019', 'B10_2019', 'B12_2019', 'mndwi_2019', 'mbi_2019', 'B1_2020', 'B2_2020', 'B6_2020', 'B7_2020', 'B9_2020', 'B10_2020', 'B12_2020', 'mndwi_2020', 'brba_2020', 'mbi_2020']


### [Baseline] Logistic Regression Results Sans Hyperparamater Optimization

In [27]:
lr = LogisticRegression()
results, clfs = model_utils.geospatialcv(data, lr_best_features, label, lr, verbose=2);


Test set: MAICAO
[[29566   434]
 [   80   630]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     30000
           1       0.59      0.89      0.71       710

    accuracy                           0.98     30710
   macro avg       0.79      0.94      0.85     30710
weighted avg       0.99      0.98      0.98     30710

MAICAO Results: 
- F1 Score: 0.7103
- Kappa Statistics: 0.7020
- Precision: 0.5921
- Recall: 0.8873
- Accuracy: 0.9833

Test set: RIOHACHA
[[29740   260]
 [  693  2808]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     30000
           1       0.92      0.80      0.85      3501

    accuracy                           0.97     33501
   macro avg       0.95      0.90      0.92     33501
weighted avg       0.97      0.97      0.97     33501

RIOHACHA Results: 
- F1 Score: 0.8549
- Kappa Statistics: 0.8392
- Precision: 0.9153
- Recall: 0.8021
- Accuracy: 0.9716

Test 

### Grid Search Cross Validation

In [28]:
param_grid = {
    'classifier__penalty': ['l2'],
    'classifier__C':[0.001, 0.01, 1.0, 5.0, 10]
}
lr_grid_search = model_utils.hyperparameter_optimization(
    data, lr_best_features, label, lr, param_grid, scoring='f1', verbose=2
)

Fitting 6 folds for each of 5 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   27.0s finished


Best Paramaters: {'classifier__C': 5.0, 'classifier__penalty': 'l2'}


### Logistic Regression Results with Hyperparameter Optimization

In [29]:
lr = LogisticRegression(C=5.0)
results, clfs = model_utils.geospatialcv(data, lr_best_features, label, lr, verbose=2);


Test set: MAICAO
[[29626   374]
 [   80   630]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     30000
           1       0.63      0.89      0.74       710

    accuracy                           0.99     30710
   macro avg       0.81      0.94      0.86     30710
weighted avg       0.99      0.99      0.99     30710

MAICAO Results: 
- F1 Score: 0.7351
- Kappa Statistics: 0.7277
- Precision: 0.6275
- Recall: 0.8873
- Accuracy: 0.9852

Test set: RIOHACHA
[[29779   221]
 [  663  2838]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     30000
           1       0.93      0.81      0.87      3501

    accuracy                           0.97     33501
   macro avg       0.95      0.90      0.93     33501
weighted avg       0.97      0.97      0.97     33501

RIOHACHA Results: 
- F1 Score: 0.8652
- Kappa Statistics: 0.8507
- Precision: 0.9278
- Recall: 0.8106
- Accuracy: 0.9736

Test 

## LinearSVC

In [30]:
lsvc = LinearSVC(random_state=SEED)
lsvc_best_features = model_utils.rfecv_feature_selection(
    lr, data, features, label, scoring='f1', step=5, verbose=0
)
print(lsvc_best_features)

['B10_2016', 'B12_2016', 'nbi_2016', 'mbi_2016', 'B1_2017', 'B2_2017', 'B8_2017', 'B10_2017', 'B12_2017', 'B11_2018', 'nbi_2018', 'B1_2019', 'B2_2019', 'B3_2019', 'B6_2019', 'B7_2019', 'B9_2019', 'B10_2019', 'B12_2019', 'savi_2019', 'mndwi_2019', 'brba_2019', 'mbi_2019', 'B1_2020', 'B2_2020', 'B3_2020', 'B6_2020', 'B7_2020', 'B8_2020', 'B9_2020', 'B10_2020', 'B12_2020', 'savi_2020', 'ui_2020', 'brba_2020', 'mbi_2020']


### [Baseline] LinearSVC Results Sans Hyperparamater Optimization

In [31]:
lsvc = LinearSVC(random_state=SEED)
results, clfs = model_utils.geospatialcv(data, lsvc_best_features, label, lsvc, verbose=2);


Test set: MAICAO
[[29697   303]
 [  118   592]]
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     30000
           1       0.66      0.83      0.74       710

    accuracy                           0.99     30710
   macro avg       0.83      0.91      0.87     30710
weighted avg       0.99      0.99      0.99     30710

MAICAO Results: 
- F1 Score: 0.7377
- Kappa Statistics: 0.7308
- Precision: 0.6615
- Recall: 0.8338
- Accuracy: 0.9863

Test set: RIOHACHA
[[29907    93]
 [ 1025  2476]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     30000
           1       0.96      0.71      0.82      3501

    accuracy                           0.97     33501
   macro avg       0.97      0.85      0.90     33501
weighted avg       0.97      0.97      0.96     33501

RIOHACHA Results: 
- F1 Score: 0.8158
- Kappa Statistics: 0.7979
- Precision: 0.9638
- Recall: 0.7072
- Accuracy: 0.9666

Test 

### Grid Search Cross Validation
Reference: https://towardsdatascience.com/svm-hyper-parameter-tuning-using-gridsearchcv-49c0bc55ce29

In [ ]:
lsvc = LinearSVC(random_state=SEED)
param_grid = {'classifier__C': [0.001, 0.1, 0.5, 1, 5, 10, 100, 1000]}
lsvc_grid_search = model_utils.hyperparameter_optimization(
    data, lsvc_best_features, label, lsvc, param_grid, scoring='f1', verbose=2
)

Fitting 6 folds for each of 8 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


### LinearSVC Results with Hyperparameter Optimization

In [ ]:
lsvc = LinearSVC(C=0.5, random_state=SEED)
results, clfs = model_utils.geospatialcv(data, lsvc_best_features, label, lsvc, verbose=2);